In [ ]:
import os
from tqdm import tqdm

import numpy as np


project_dir = os.path.dirname(os.path.abspath(""))

data_path = "/home/S22/workspace/BLAST/sampling/output_p_linear/final_result.dat"


In [ ]:
shape = (20_000_000, 4096)
data = np.memmap(data_path, dtype=np.float32, mode='r+', shape=shape)
shape

In [ ]:
# 将全是NaN的行设置成全0
all_nan_count = 0
for i in tqdm(range(shape[0])):
    if np.isnan(data[i]).all():
        data[i] = 0
        all_nan_count += 1
print(f'All nan count: {all_nan_count}')


In [ ]:
# 去掉每一行左侧的NaN
processed_count = 0
for i in tqdm(range(shape[0])):
    # 找到左侧第一个不是NaN的索引
    first_non_nan_index = np.argmax(~np.isnan(data[i]))
    if first_non_nan_index > 0:
        # 去掉左侧的NaN
        processed_count += 1
        data[i] = np.concatenate([data[i][first_non_nan_index:], np.full((first_non_nan_index,), np.nan)])
print(f'Processed count: {processed_count}')

In [ ]:
# 将每一行的中间的NaN值替换为0，后面的NaN值保留
processed_count = 0
for i in tqdm(range(shape[0])):
    # 找到中间第一个不是NaN的索引
    first_non_nan_index = np.argmax(~np.isnan(data[i]))
    # 找到中间最后一个不是NaN的索引
    last_non_nan_index = shape[1] - np.argmax(np.flipud(~np.isnan(data[i])))
    assert first_non_nan_index < last_non_nan_index
    # 去掉中间的NaN
    processed_count += 1
    data[i][first_non_nan_index:last_non_nan_index] = np.nan_to_num(data[i][first_non_nan_index:last_non_nan_index], nan=0, posinf=0, neginf=0)
print(f'Processed count: {processed_count}')

In [ ]:
# test
import numpy as np
test_array = np.array([1, 2, 3, np.nan, 4, np.nan, np.nan])
last_non_nan_index = test_array.shape[0] - np.argmax(np.flipud(~np.isnan(test_array)))
last_non_nan_index